In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('../optimization_results2.csv')

In [30]:
df

,Problem ID,Function ID,Instance ID,Algorithm,Best Solution (X),Best Function Value (F)
0,bbob_f001_i01_d20,1,1,PSO,"[0.25271295628667456, -1.1567240917649382, -0....",79.480001
1,bbob_f001_i01_d20,1,1,CMAES,"[0.2527999722638068, -1.1567999681501844, -0.7...",79.480000
2,bbob_f001_i01_d20,1,1,DE,"[0.23997986691433615, -1.173100748697348, -0.7...",79.493205
3,bbob_f001_i01_d20,1,1,L-BFGS-B,"[0.25280004924197097, -1.156800450762462, -0.7...",79.480000
4,bbob_f001_i01_d20,1,1,BIPOP-CMAES,"[0.2527999680449557, -1.1567999946809069, -0.7...",79.480000
...,...,...,...,...,...,...
5995,bbob_f024_i50_d20,24,50,PSO,"[-0.0059106110868235116, 1.8482787388272481, -...",-32.369769
5996,bbob_f024_i50_d20,24,50,CMAES,"[-1.6860998289527105, 0.49009201787828616, -2....",-74.701640
5997,bbob_f024_i50_d20,24,50,DE,"[-3.2333051272372413, 0.4093956374118155, -2.9...",37.438061
5998,bbob_f024_i50_d20,24,50,L-BFGS-B,"[-0.49623548100467374, 3.8010394677985273, 1.2...",377.084196


In [25]:
aggregated_df = df.groupby(["Function ID", "Algorithm"]).agg(
    {"Best Function Value (F)": ["mean", "std", "min", "max"]}
)
aggregated_df


Best Function Value (F)                           \
                                           mean         std          min   
Function ID Algorithm                                                      
1           BIPOP-CMAES                7.397800  265.170920 -1000.000000   
            CMAES                      7.397800  265.170920 -1000.000000   
            DE                         7.407956  265.171669  -999.994440   
            L-BFGS-B                   7.397800  265.170920 -1000.000000   
            PSO                        7.397802  265.170919  -999.999996   
...                                         ...         ...          ...   
24          BIPOP-CMAES              -15.293956  329.607767  -959.891099   
            CMAES                    -21.019558  323.832971  -937.822189   
            DE                        76.669009  321.804637  -830.822325   
            L-BFGS-B                 392.045751  330.891043  -486.495387   
            PSO                       52.207406  320.137677  -859.146405   

                                      
                                 max  
Function ID Algorithm                 
1           BIPOP-CMAES  1000.000000  
            CMAES        1000.000000  
            DE           1000.012585  
            L-BFGS-B     1000.000000  
            PSO          1000.000001  
...                              ...  
24          BIPOP-CMAES   796.677772  
            CMAES         880.782481  
            DE            918.308925  
            L-BFGS-B     1455.038704  
            PSO           910.183557  

[120 rows x 4 columns]

In [26]:
aggregated_df.columns = ["_".join(col).strip() for col in aggregated_df.columns.to_flat_index()]
aggregated_df.reset_index(inplace=True)
aggregated_df["Rank"] = aggregated_df.groupby("Function ID")["Best Function Value (F)_mean"].rank(
     ascending=True
)

In [27]:
aggregated_df

,Function ID,Algorithm,Best Function Value (F)_mean,Best Function Value (F)_std,Best Function Value (F)_min,Best Function Value (F)_max,Rank
0,1,BIPOP-CMAES,7.397800,265.170920,-1000.000000,1000.000000,1.0
1,1,CMAES,7.397800,265.170920,-1000.000000,1000.000000,2.0
2,1,DE,7.407956,265.171669,-999.994440,1000.012585,5.0
3,1,L-BFGS-B,7.397800,265.170920,-1000.000000,1000.000000,3.0
4,1,PSO,7.397802,265.170919,-999.999996,1000.000001,4.0
...,...,...,...,...,...,...,...
115,24,BIPOP-CMAES,-15.293956,329.607767,-959.891099,796.677772,2.0
116,24,CMAES,-21.019558,323.832971,-937.822189,880.782481,1.0
117,24,DE,76.669009,321.804637,-830.822325,918.308925,4.0
118,24,L-BFGS-B,392.045751,330.891043,-486.495387,1455.038704,5.0


In [33]:
def calculate_scores(group):
    group["Best Function Value (F)"] = group["Best Function Value (F)"].fillna(float('inf'))
    group["Score"] = group["Best Function Value (F)"].rank(method="dense", ascending=True).astype(int)
    group["Score"] = group["Score"].max() - group["Score"] + 1
    return group

df = df.groupby("Problem ID").apply(calculate_scores)

result = df.pivot(index="Problem ID", columns="Algorithm", values="Score").reset_index()
result.fillna(0, inplace=True)

result.columns.name = None
result

,Problem ID,BIPOP-CMAES,CMAES,DE,L-BFGS-B,PSO
0,bbob_f001_i01_d20,5,4,1,3,2
1,bbob_f001_i02_d20,4,4,1,3,2
2,bbob_f001_i03_d20,4,4,1,3,2
3,bbob_f001_i04_d20,5,4,1,3,2
4,bbob_f001_i05_d20,5,4,1,3,2
...,...,...,...,...,...,...
1195,bbob_f024_i46_d20,5,4,3,1,2
1196,bbob_f024_i47_d20,5,4,2,1,3
1197,bbob_f024_i48_d20,5,4,2,1,3
1198,bbob_f024_i49_d20,4,5,2,1,3
